In [8]:
import cv2
from ultralytics import YOLO

# 加载YOLOv8模型
model = YOLO("yolov8m.pt")  # 使用预训练的YOLOv8n模型

# 定义车辆类别（基于COCO数据集的类别ID）
vehicle_classes = [2, 3, 5, 7]  # 2=car, 3=motorcycle, 5=bus, 7=truck

# 强光抑制函数：使用CLAHE（对比度受限的自适应直方图均衡化）
def apply_clahe(frame):
    """
    应用CLAHE来增强图像对比度，减少夜晚强光（如车灯）的影响。
    """
    # 转换到LAB颜色空间
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    
    # 对L通道应用CLAHE
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(4, 4))
    l_clahe = clahe.apply(l)
    
    # 合并通道并转换回BGR
    lab_clahe = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

# 车辆计数类
class VehicleCounter:
    def __init__(self, line_position):
        """
        初始化车辆计数器。
        :param line_position: 计数线的y坐标
        """
        self.line_position = line_position
        self.count = 0
        self.counted_ids = set()  # 存储已计数的跟踪ID，避免重复计数

    def update(self, tracks):
        """
        根据跟踪结果更新车辆计数。
        :param tracks: 检测和跟踪的目标列表
        """
        for track in tracks:
            if track['class'] in vehicle_classes:  # 只处理车辆类别
                y = track['bbox'][1]  # 获取bounding box顶部y坐标
                track_id = track['track_id']
                # 当车辆越过计数线且未被计数时，计数加一
                if y > self.line_position and track_id not in self.counted_ids:
                    self.count += 1
                    self.counted_ids.add(track_id)

# 主函数
def main(video_path):
    """
    主函数：处理视频流，进行目标检测、跟踪和车辆计数。
    :param video_path: 视频文件路径
    """
    # 打开视频流
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("无法打开视频流或文件")
        return

    # 获取视频参数
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 设置计数线位置（例如视频高度的中间）
    line_position = 860#height // 2
    counter = VehicleCounter(line_position)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 应用强光抑制
        frame = apply_clahe(frame)

        # 使用YOLOv8的track方法进行目标检测和跟踪
        results = model.track(frame, persist=True, device='cpu',verbose=False  )  # persist=True保持跟踪状态，cuda加速

        # 解析跟踪结果
        tracks = []
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # 边界框坐标
                conf = box.conf[0].cpu().numpy()  # 置信度
                cls = int(box.cls[0].cpu().numpy())  # 类别ID
                track_id = int(box.id[0].cpu().numpy()) if box.id is not None else -1  # 跟踪ID
                tracks.append({
                    'bbox': [x1, y1, x2, y2],
                    'conf': conf,
                    'class': cls,
                    'track_id': track_id
                })

        # 更新车辆计数
        counter.update(tracks)

        # 可视化：绘制检测框、跟踪ID、计数线和计数值
        for track in tracks:
            if track['class'] in vehicle_classes:
                x1, y1, x2, y2 = map(int, track['bbox'])
                track_id = track['track_id']
                # 绘制检测框
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # 标注跟踪ID
                cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # 绘制计数线
        cv2.line(frame, (0, line_position), (width, line_position), (0, 0, 255), 2)
        # 显示计数值
        cv2.putText(frame, f'Count: {counter.count}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # 显示处理后的帧
        cv2.imshow('Highway Vehicle Counting', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # 按'q'键退出
            break

    # 释放资源
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "highway_n.avi"  # 请替换为实际视频路径
    main(video_path)